**Assignment 2**<br>
Name : Jayant<br>
Roll Number: 102303825 <br>
Sub-group: 2C61


**Part I : Based on Feature Selection, Cleaning, and Preprocessing to Construct an Input from Data Source.**

(a) Examine the values of each attribute and Select a set of attributes only that would affect to predict
future bike buyers to create your input for data mining algorithms. Remove all the unnecessary
attributes. (Select features just by analysis).

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,KBinsDiscretizer,OneHotEncoder,LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
df_original=pd.read_csv("AWCustomers.csv")
print(df_original.shape)
print(df_original.info())
print(df_original.describe(include='all'))

selected_features = [
    'BirthDate',
    'Gender',
    'MaritalStatus',
    'YearlyIncome',
    'HomeOwnerFlag',
    'NumberCarsOwned',
    'TotalChildren',
    'Education',
    'Occupation'
]
selected_df = df_original[selected_features].copy()
print(selected_df.shape)
print(selected_df.info())
print(selected_df.describe(include='all'))

(18361, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerID            18361 non-null  int64 
 1   Title                 101 non-null    object
 2   FirstName             18361 non-null  object
 3   MiddleName            10572 non-null  object
 4   LastName              18361 non-null  object
 5   Suffix                3 non-null      object
 6   AddressLine1          18361 non-null  object
 7   AddressLine2          311 non-null    object
 8   City                  18361 non-null  object
 9   StateProvinceName     18361 non-null  object
 10  CountryRegionName     18361 non-null  object
 11  PostalCode            18361 non-null  object
 12  PhoneNumber           18361 non-null  object
 13  BirthDate             18361 non-null  object
 14  Education             18361 non-null  object
 15  Occupation            18

(b) Create a new Data Frame with the selected attributes only.


In [9]:
df=pd.DataFrame(selected_df)
print(df.head())

    BirthDate Gender MaritalStatus  YearlyIncome  HomeOwnerFlag  \
0  1987-11-13      M             M         81916              1   
1  1972-07-21      M             M         81076              1   
2  1985-11-09      F             S         86387              0   
3  1977-10-18      M             M         61481              1   
4  1975-02-05      M             S         51804              1   

   NumberCarsOwned  TotalChildren        Education      Occupation  
0                3              1        Bachelors        Clerical  
1                2              2  Partial College        Clerical  
2                3              0        Bachelors        Clerical  
3                2              2  Partial College  Skilled Manual  
4                1              0  Partial College  Skilled Manual  


(c) Determine a Data value type (Discrete, or Continuous, then Nominal, Ordinal, Interval, Ratio) of
each attribute in your selection to identify preprocessing tasks to create input for your data mining.


In [10]:
data_types = {
        'BirthDate': 'Continuous-date',           # (convert)
        'Gender': 'Nominal',                      # (One-Hot)
        'MaritalStatus': 'Nominal',               # (One-Hot)
        'YearlyIncome': 'Continuous-ratio',       # (Standardize and Normalize)
        'HomeOwnerFlag': 'Binary',                # (keep)
        'NumberCarsOwned': 'Discrete-ratio',      # (Standardize)
        'TotalChildren': 'Discrete-ratio',        # (Standardize)
        'Education': 'Ordinal',                   # (Label Encode)
        'Occupation': 'Nominal'                   # (One-Hot)
      }
for col,dtype in data_types.items():
  print(f"{col}: {dtype}")

BirthDate: Continuous-date
Gender: Nominal
MaritalStatus: Nominal
YearlyIncome: Continuous-ratio
HomeOwnerFlag: Binary
NumberCarsOwned: Discrete-ratio
TotalChildren: Discrete-ratio
Education: Ordinal
Occupation: Nominal


**Part II: Data Preprocessing and Transformation**

Depending on the data type of each attribute, transform each object from your preprocessed data.
Use all the data rows (~= 18000 rows) with the selected features as input to apply all the tasks below, do
not perform each task on the smaller data set that you got from your random sampling result.


(a) Handling Null values

(b) Normalization

(c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different values

(d) Standardization/Normalization

(e) Binarization (One Hot Encoding)

In [11]:
new_df = df.copy()
# print(df.isnull().sum())

# 1. Convert BirthDate to Age (simple conversion)
new_df['Age'] = (pd.to_datetime('today') - pd.to_datetime(new_df['BirthDate'],errors='coerce')).dt.days // 365
new_df.drop('BirthDate',axis=1,inplace=True)

# 2. Normalize Age to [0,1] range
new_df['Age'] = MinMaxScaler().fit_transform(new_df[['Age']])

# 3. One-hot encode Gender (transform to 0/1)
gender_dummies = pd.get_dummies(new_df['Gender'],prefix='Gender',drop_first=True)
new_df = pd.concat([new_df.drop('Gender',axis=1),gender_dummies],axis=1)

# 4. One-hot encode MaritalStatus (transform to 0/1)
marital_dummies = pd.get_dummies(new_df['MaritalStatus'],prefix='MaritalStatus',drop_first=True)
new_df = pd.concat([new_df.drop('MaritalStatus',axis=1),marital_dummies],axis=1)

# 5. Discretize YearlyIncome into 5 bins
binner = KBinsDiscretizer(n_bins=5,encode='ordinal',strategy='quantile')
new_df['YearlyIncomeBinned'] = binner.fit_transform(new_df[['YearlyIncome']])

# 6. Standardize YearlyIncome (original continuous version)
new_df['YearlyIncome'] = StandardScaler().fit_transform(new_df[['YearlyIncome']])

# 7. Standardize NumberCarsOwned
new_df['NumberCarsOwned'] = StandardScaler().fit_transform(new_df[['NumberCarsOwned']])

# 8. Standardize TotalChildren
new_df['TotalChildren'] = StandardScaler().fit_transform(new_df[['TotalChildren']])

# 9. Label encode Education
new_df['Education'] = LabelEncoder().fit_transform(new_df['Education'])

# 10. One-hot encode Occupation
occupation_dummies = pd.get_dummies(new_df['Occupation'], prefix='Occupation', drop_first=True)
new_df = pd.concat([new_df.drop('Occupation', axis=1), occupation_dummies], axis=1)

# Display the transformed DataFrame
print("\nTransformed DataFrame (first 5 rows):")
print(new_df.head())


Transformed DataFrame (first 5 rows):
   YearlyIncome  HomeOwnerFlag  NumberCarsOwned  TotalChildren  Education  \
0      0.298555              1         1.892524       0.161342          0   
1      0.271180              1         0.798389       1.239753          3   
2      0.444261              0         1.892524      -0.917069          0   
3     -0.367401              1         0.798389       1.239753          3   
4     -0.682765              1        -0.295746      -0.917069          3   

        Age  Gender_M  MaritalStatus_S  YearlyIncomeBinned  \
0  0.183099      True            False                 3.0   
1  0.408451      True            False                 3.0   
2  0.211268     False             True                 3.0   
3  0.323944      True            False                 2.0   
4  0.366197      True             True                 1.0   

   Occupation_Management  Occupation_Manual  Occupation_Professional  \
0                  False              False          

**Part III: Calculating Proximity /Correlation Analysis of two features**

Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each
nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct
similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively.

(a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two following objects of your transformed input data.

In [12]:
# Pick two sample objects (rows) from the transformed dataset
obj1 = new_df.iloc[0]
obj2 = new_df.iloc[1]

# Simple Matching Coefficient (SMC)
def simple_matching(x,y):
    return np.sum(x == y)/len(x)

smc = simple_matching(obj1,obj2)

# Jaccard Similarity (only for binary attributes)
def jaccard_similarity(x,y):
    intersection = np.sum((x==1)&(y==1))
    union = np.sum((x==1)|(y==1))
    return intersection/union if union != 0 else 0

# We restrict Jaccard to one-hot encoded / binary columns
binary_cols = [col for col in new_df.columns if set(new_df[col].unique()).issubset({0,1})]
jac = jaccard_similarity(new_df[binary_cols].iloc[0],new_df[binary_cols].iloc[1])

# Cosine Similarity
cos_sim = cosine_similarity([obj1],[obj2])[0][0]

print("Simple Matching Coefficient:", smc)
print("Jaccard Similarity:", jac)
print("Cosine Similarity:", cos_sim)

Simple Matching Coefficient: 0.6153846153846154
Jaccard Similarity: 1.0
Cosine Similarity: 0.7081006666266153


(b) Calculate Correlation between two features Commute Distance and Yearly Income

In [13]:
# Correlation between CommuteDistance and YearlyIncome
if 'CommuteDistance' in df_original.columns and 'YearlyIncome' in df_original.columns:
    correlation = df_original['CommuteDistance'].astype('category').cat.codes.corr(df_original['YearlyIncome'])
    print("Correlation between Commute Distance and Yearly Income:", correlation)
else:
    print("CommuteDistance column not found in dataset.")

CommuteDistance column not found in dataset.
